In [1]:
import sqlite3
from rich.console import Console
from rich.table import Table
from rich.style import Style
from rich.text import Text

def query_payment_data(cursor, params):
    """
    查询支付数据
    
    Args:
        cursor: 数据库游标
        params: 查询参数字典
    
    Returns:
        tuple: (success, result)
        - success: bool, 查询是否成功
        - result: 成功时为数据列表，失败时为错误信息
    """
    try:
        # 检查表是否存在
        table_name = f"pay_{params['monthyear']}"
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' AND name=?
        """, (table_name,))
        
        if not cursor.fetchone():
            return False, f"表格 {table_name} 不存在"
        
        # 构建查询条件
        conditions = []
        query_params = []
        
        # 标签筛选
        # if params['tag'] and params['tag'] != "N":
        if params['tag'] :
            conditions.append("pay_tag = ?")
            query_params.append(params['tag'])
        
        # 关键词搜索
        if params['key']:
            conditions.append("(pay_source LIKE ? OR pay_note LIKE ?)")
            query_params.extend([f"%{params['key']}%", f"%{params['key']}%"])
        
        # 金额范围筛选
        if params['lt']:
            try:
                lt_value = float(params['lt'])
                conditions.append("pay_money < ?")
                query_params.append(lt_value)
            except ValueError:
                return False, "金额上限格式错误"
        
        if params['gt']:
            try:
                gt_value = float(params['gt'])
                conditions.append("pay_money > ?")
                query_params.append(gt_value)
            except ValueError:
                return False, "金额下限格式错误"
        
        # 构建完整的SQL查询
        query = f"SELECT * FROM {table_name}"
        if conditions:
            query += " WHERE " + " AND ".join(conditions)
        query += " ORDER BY pay_money DESC"
        
        # 执行查询
        cursor.execute(query, query_params)
        
        # 获取列名
        columns = [description[0] for description in cursor.description]
        
        # 将结果转换为字典列表
        results = []
        for row in cursor.fetchall():
            results.append(dict(zip(columns, row)))
        
        return True, results
        
    except sqlite3.Error as e:
        return False, f"数据库错误: {str(e)}"

def display_query_results(results, params):
    """
    使用rich库显示查询结果
    """
    console = Console()
    
    if not results:
        console.print("[red]没有找到数据[/red]")
        return
    
    # 创建表格
    table = Table(
        title=f"支付记录 ({params['monthyear']})",
        show_header=True,
        header_style="bold magenta",
        border_style="blue",
        title_style="bold cyan"
    )
    
    # 添加列
    # table.add_column("ID", style="dim")
    table.add_column("日期", style="cyan")
    table.add_column("金额", justify="right", style="green")
    table.add_column("来源", style="yellow")
    table.add_column("标签", style="red")
    table.add_column("备注")
    
    # 添加数据行
    for row in results:
        money = f"¥{float(row['pay_money']):.2f}"
        
        # 根据金额设置不同样式
        if float(row['pay_money']) > 100:
            money_style = "[bold red]" + money + "[/bold red]"
        else:
            money_style = "[green]" + money + "[/green]"
            
        table.add_row(
            # str(row['id']),
            row['pay_time'],
            money_style,
            row['pay_source'],
            row['pay_tag'],
            row.get('pay_note', '')
        )
    
    # 显示查询条件
    console.print("\n[bold yellow]查询条件:[/bold yellow]")
    conditions = []
    if params['tag'] and params['tag'] != "N":
        conditions.append(f"标签: {params['tag']}")
    if params['key']:
        conditions.append(f"关键词: {params['key']}")
    if params['lt']:
        conditions.append(f"金额 < {params['lt']}")
    if params['gt']:
        conditions.append(f"金额 > {params['gt']}")
    
    if conditions:
        console.print(" | ".join(conditions))
    
    # 打印表格
    console.print("\n")
    console.print(table)
    
    # 打印统计信息
    total_amount = sum(float(row['pay_money']) for row in results)
    console.print(f"\n[bold green]总金额: ¥{total_amount:.2f}[/bold green]")
    console.print(f"[dim]共 {len(results)} 条记录[/dim]")

# 使用示例
def main():
    # 测试参数
    params = {
        "monthyear": "202501",
        "tag": "",
        "key": "",
        "lt": "",
        "gt": ""
    }
    
    # 创建数据库连接
    # conn = sqlite3.connect('moneycount.db')
    cursor = conn.cursor()
    
    # 执行查询
    success, result = query_payment_data(cursor, params)
    
    if success:
        display_query_results(result, params)
    else:
        console = Console()
        console.print(f"[red]错误: {result}[/red]")
    
    # 关闭连接
    conn.close()

if __name__ == "__main__":
    main()

NameError: name 'conn' is not defined